In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def toUpperNoSpaces(row,column):
    return str(row[column]).replace(" ","").upper().strip()
def BoxOfficeNumber(row,column):
    return float(str(row[column]).replace("$","").replace(",","").strip())
data_folder = "C:/Users/Deepika Chandrababu/Desktop/Infoviz/dataset/"

dfAll = pd.read_csv(data_folder + "netflix-rotten-tomatoes-metacritic-imdb.csv")
# dfOM = pd.read_csv('netflixOM.csv')
# dfOS = pd.read_csv('netflixOS.csv')
dfOS = pd.read_csv(data_folder + "NetflixOriginals.csv")
dfOM = pd.read_csv(data_folder + "netflix_original_movie_data.csv")

# Adding new columns for operations like: compare between datasets
dfOM['TitleToCompare'] = dfOM.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)
dfOS['TitleToCompare'] = dfOS.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)
dfAll['TitleToCompare'] = dfAll.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)

dfAll['GenreToCompare'] = dfAll.apply(lambda row: toUpperNoSpaces(row,'Genre'), axis = 1)
dfAll['BoxOfficeProfits'] = dfAll.apply(lambda row: BoxOfficeNumber(row,'Boxoffice'), axis = 1)

# add new column: Original, 1: is produced by netflix, 0: is not produced by netflix
df = dfAll.assign(Original = dfAll.TitleToCompare.isin(pd.concat([dfOM.TitleToCompare,dfOS.TitleToCompare])).astype(int))
df['Year'] = pd.to_numeric(df['Release Date'].str[-4:])

#filtering first writer
df['First_Writer'] = df.Writer.str.split(', ').str[0]
df.head()

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,...,Image,Poster,TMDb Trailer,Trailer Site,TitleToCompare,GenreToCompare,BoxOfficeProfits,Original,Year,First_Writer
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,...,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN,LETSFIGHTGHOST,"CRIME,DRAMA,FANTASY,HORROR,ROMANCE",2122065.0,0,2008.0,John Ajvide Lindqvist
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,...,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube,HOWTOBUILDAGIRL,COMEDY,70632.0,0,2020.0,Caitlin Moran
2,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,"Brendan Walsh, Daley Nixon",...,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube,CENTIGRADE,"DRAMA,THRILLER",16263.0,0,2020.0,Brendan Walsh
3,ANNE+,Drama,"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"Belgium,Netherlands",< 30 minutes,NaN,NaN,...,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN,ANNE+,DRAMA,NaN,0,2016.0,NaN
4,Moxie,"Animation, Short, Drama","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,"Lithuania,Poland,France,Iceland,Italy,Spain,Gr...",1-2 hour,Stephen Irwin,NaN,...,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN,MOXIE,"ANIMATION,SHORT,DRAMA",NaN,1,2011.0,NaN


In [11]:
df['First_Writer'] 

0        John Ajvide Lindqvist
1                Caitlin Moran
2                Brendan Walsh
3                          NaN
4                          NaN
                 ...          
15475                      NaN
15476                      NaN
15477                      NaN
15478                      NaN
15479                      NaN
Name: First_Writer, Length: 15480, dtype: object

In [14]:
dfTrend = pd.DataFrame()
# get unique values from Genre
dfTemp = df['GenreToCompare'].drop_duplicates().str.split(',', expand=True)
dfGenre = dfTemp[0].drop_duplicates()
for col in dfTemp.columns:
    dfGenre = dfGenre.append(dfTemp[col].drop_duplicates(), ignore_index=True)

# drop duplicates and "Nan" value from Genre unique values 
dfGenre = dfGenre.drop_duplicates().dropna().drop(dfGenre.index[10])



filterYear = df['Year'] >= 2010
for genre in dfGenre:
    filterGenre = df['GenreToCompare'].str.contains(pat = genre)
    dfTrend = dfTrend.append(df[['Original','Writer','Series or Movie','Year','GenreToCompare','IMDb Score','Title','First_Writer']].where(filterYear & filterGenre, inplace = False).dropna().drop_duplicates().groupby(['Year','Original','Series or Movie','Writer','Title','First_Writer']).agg(Films = ('Title', 'count'),Writers = ('Writer','count'),Rating = ('IMDb Score', 'mean')).assign(Genre = genre)).sort_values(by=['Rating'],ascending=False)

dfTrend = dfTrend.reset_index()


#export to csv file

# dfTrend.sort_values(by=['Rating'],ascending=False) 
# dfTrend.sort_values(['Genre','Year','Original','Series or Movie','Writer','Title','First_Writer','Rating'], ascending=[True,True,True,True,True,True,True,False], inplace=True)
dfTrend

,Year,Original,Series or Movie,Writer,Title,First_Writer,Films,Writers,Rating,Genre
0,2020.0,0.0,Movie,Andy Daniluc,No Festival,Andy Daniluc,1,1,9.7,MUSIC
1,2020.0,0.0,Movie,Andy Daniluc,No Festival,Andy Daniluc,1,1,9.7,DOCUMENTARY
2,2013.0,0.0,Series,"Justin Roiland, Dan Harmon",Rick and Morty,Justin Roiland,1,1,9.2,ANIMATION
3,2021.0,0.0,Series,Earl Crowe,Im Sorry,Earl Crowe,1,1,9.2,DOCUMENTARY
4,2013.0,0.0,Series,"Justin Roiland, Dan Harmon",Rick and Morty,Justin Roiland,1,1,9.2,COMEDY
...,...,...,...,...,...,...,...,...,...,...
19341,2016.0,0.0,Movie,"Maciej Pawlicki, Tomasz Lysiak, Marcin Wolski,...",Smoleńsk,Maciej Pawlicki,1,1,1.4,THRILLER
19342,2020.0,0.0,Movie,Abir Sengupta,Indoo Ki Jawani,Abir Sengupta,1,1,1.4,COMEDY
19343,2019.0,0.0,Movie,"Hasan Karakus, Hande Turan",Debt Fees,Hasan Karakus,1,1,1.4,COMEDY
19344,2016.0,0.0,Movie,"Maciej Pawlicki, Tomasz Lysiak, Marcin Wolski,...",Smoleńsk,Maciej Pawlicki,1,1,1.4,DRAMA


In [18]:
dfTrend.to_csv(data_folder + 'writer_fulldata.csv', index = False, header=True)

In [15]:
# top 15 writers
top_writers_based_on_rating =dfTrend[0:15]
top_writers_based_on_rating

,Year,Original,Series or Movie,Writer,Title,First_Writer,Films,Writers,Rating,Genre
0,2020.0,0.0,Movie,Andy Daniluc,No Festival,Andy Daniluc,1,1,9.7,MUSIC
1,2020.0,0.0,Movie,Andy Daniluc,No Festival,Andy Daniluc,1,1,9.7,DOCUMENTARY
2,2013.0,0.0,Series,"Justin Roiland, Dan Harmon",Rick and Morty,Justin Roiland,1,1,9.2,ANIMATION
3,2021.0,0.0,Series,Earl Crowe,Im Sorry,Earl Crowe,1,1,9.2,DOCUMENTARY
4,2013.0,0.0,Series,"Justin Roiland, Dan Harmon",Rick and Morty,Justin Roiland,1,1,9.2,COMEDY
5,2013.0,0.0,Series,"Justin Roiland, Dan Harmon",Rick and Morty,Justin Roiland,1,1,9.2,ADVENTURE
6,2013.0,0.0,Series,"Justin Roiland, Dan Harmon",Rick and Morty,Justin Roiland,1,1,9.2,SCI-FI
7,2010.0,0.0,Series,"Steven Moffat, Mark Gatiss",Sherlock,Steven Moffat,1,1,9.1,THRILLER
8,2014.0,0.0,Series,Juliano Enrico,Irmão do Jorel,Juliano Enrico,1,1,9.1,ANIMATION
9,2010.0,0.0,Series,"Steven Moffat, Mark Gatiss",Sherlock,Steven Moffat,1,1,9.1,CRIME
